***Jusqu'ici l'espace de sortie était fixé à [0,9], ce qui engendrait des errerus de choix d'actions impossibles, et donc des décisions randoms. Je veux éviter cela désormais, on va créer un masque pour n'afficher que les solutions possibles, et si possible ce qu'elles représentent. On passe en sortie dynamique***

In [25]:
#1 on se place 
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [26]:
from Utils.embedding.type.type_embedding import *
from Players.Player4 import *
from Utils.dynamic_output.dynamic_output import *

In [27]:
############## DEFINIR L'EMBEDDING ##################
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen4EnvSinglePlayer
from poke_env.data import GenData
from poke_env.environment.abstract_battle import AbstractBattle
import torch
from gymnasium.spaces import Discrete, Box
import random

# Initialiser GenData pour la génération souhaitée (par exemple, génération 4)
gen_data = GenData.from_gen(4)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class Player4TrainingDynamic(Gen4EnvSinglePlayer): #Classe pour entrainer le modèle 4. 
    def __init__(self,model_path=None, **kwargs):
        super().__init__(**kwargs)
        if model_path is not None :
            #self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
            print(f"📥 Modèle chargé depuis {model_path}")

        self.action_space = Discrete(9)  # ✅ attribut classique
        self.my_team = None
        self.my_team_index = {}
        self.opponent_team = None
        self.opponent_team_index = {}
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle)  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        if battle.turn == 1:
            self.my_team, self.my_team_index = init_my_team(battle)
            self.opponent_team, self.opponent_team_index = init_opponent_team(battle)
            print("my_team_index au tour 1 : ",self.my_team_index)
        else : 
            if battle.active_pokemon is None or battle.active_pokemon.fainted or hasattr(self, 'last_active_pokemon') and self.last_active_pokemon != battle.active_pokemon:
                self.my_team = update_my_team(battle, self.my_team, self.my_team_index)
            
            self.opponent_team, self.opponent_team_index = update_opponent_team(battle, self.opponent_team, self.opponent_team_index)
            
            self.last_active_pokemon = battle.active_pokemon

        _,action_mask,_ = get_valid_actions(battle)

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                action_mask,
                moves_real_power,
                self.my_team,
                self.opponent_team
            ]
        )
        final_tensor = torch.tensor(final_vector,dtype=torch.float32)
        #print("taille tensor state : ",final_tensor.shape)
        #print("index mon équipe : ",self.my_team_index)
        #print("index equipe adverse : ",self.opponent_team_index)
        return final_tensor

    def describe_embedding(self) -> Space:
        low = (
            [0]* 9 +            # action_mask
            [-1] * 4 +          # real power
            [0] * 72 +         # my team types
            [0] * 72           # opponent team types
        )
        high = (
            [1] * 9 +           # action_mask
            [3] * 4 +           # real power
            [1] * 72 +         # my team types
            [1] * 72           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        valid_action_ids, _,action_ids_ref_dict = get_valid_actions(battle)
        print(action_ids_ref_dict)
        print("my_team index après : ",self.my_team_index)
        #if not valid_action_ids :
        #    print("pas de moves disponibles -> choix aléatoire")
        #    action = self.choose_random_move(battle)

        #elif action not in valid_action_ids:
        #    print("acion hors moves disponibles")
        #    action = random.choice(valid_action_ids)
        #    action = super().action_to_move(action, battle)

        #else : 
            #print("action choisie/voulue par le DQN :",action_ids_ref_dict[action])

        action = super().action_to_move(action, battle)
        print("action réelle effectuée :", action)
        return action

In [28]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à deux couche cachée, sortie linéaire, f activation = Relu,
class DQNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)

In [29]:

#opponent = AdversarialTrainPlayer(battle_format="gen4randombattle")
from stable_baselines3.common.vec_env import DummyVecEnv

# Réinstancier l'env
train_env_raw = Player4TrainingDynamic(
    battle_format="gen4randombattle",
    opponent=RandomPlayer(battle_format="gen4randombattle"),
    start_challenging=True
)

train_env = DummyVecEnv([lambda: train_env_raw])  # Redéfini bien après reset

Task exception was never retrieved
future: <Task finished name='Task-359' coro=<PSClient._handle_message() done, defined at /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/ps_client/ps_client.py:122> exception=KeyError('manectric')>
Traceback (most recent call last):
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/ps_client/ps_client.py", line 191, in _handle_message
    raise exception
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/ps_client/ps_client.py", line 136, in _handle_message
    await self._handle_battle_message(split_messages)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/s

In [30]:
env = train_env

model = DQN(
    policy=MaskedDQNPolicy,
    env=env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=1,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[128, 64])
)

my_team_index au tour 1 :  {'arceuspsychic': 0}


In [31]:
steps = 2
callback = CustomTQDMCallback(total_timesteps = steps, check_freq=1000, verbose=0)
model.learn(
    total_timesteps=steps,
    callback=callback
)
model.save("Players/player_4_dynamic")

📊 Training progress:   0%|          | 0/2 [00:00<?, ?it/s]

2025-04-14 18:32:14,603 - Player4TrainingD 4 - ERROR - Unhandled exception raised while handling message:
>battle-gen4randombattle-36390
|
|t:|1744648334
|switch|p1a: Jumpluff|Jumpluff, L91, M|284/284
|move|p2a: Gyarados|Earthquake|p1a: Jumpluff
|-immune|p1a: Jumpluff
|
|-heal|p2a: Gyarados|85/100|[from] item: Leftovers
|upkeep
|turn|4
Traceback (most recent call last):
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/ps_client/ps_client.py", line 136, in _handle_message
    await self._handle_battle_message(split_messages)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/player/player.py", line 361, in _handle_battle_message
    await self._handle_battle_request(battle)
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_p

{1: ('move', hiddenpowerice (Move object)), 4: 'switch to jumpluff', 5: 'switch to weezing', 6: 'switch to abomasnow', 7: 'switch to dewgong', 8: 'switch to arceuspsychic'}
my_team index après :  {'arceuspsychic': 0}
action réelle effectuée : /choose switch jumpluff
{1: ('move', hiddenpowerice (Move object)), 4: 'switch to jumpluff', 5: 'switch to weezing', 6: 'switch to abomasnow', 7: 'switch to dewgong', 8: 'switch to arceuspsychic'}
my_team index après :  {'arceuspsychic': 0}
action réelle effectuée : /choose switch weezing


In [32]:
class Player4EvalvsBot_Dynamic(Gen4EnvSinglePlayer):

    def __init__(self, model_path="Players/player_4_dynamic", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")
        print("Répertoire actuel :", os.getcwd())
        self.my_team = None
        self.my_team_index = {}
        self.opponent_team = None
        self.opponent_team_index = {}
        
        #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle)  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        if battle.turn == 1:
            self.my_team, self.my_team_index = init_my_team(battle)
            self.opponent_team, self.opponent_team_index = init_opponent_team(battle)
        else : 
            self.my_team = update_my_team(battle, self.my_team, self.my_team_index)
            self.opponent_team, self.opponent_team_index = update_opponent_team(battle, self.opponent_team, self.opponent_team_index)

        _,action_mask = get_valid_actions(battle)

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                action_mask,
                moves_real_power,
                self.my_team,
                self.opponent_team
            ]
        )
        final_tensor = torch.tensor(final_vector,dtype=torch.float32)
        return final_tensor

    def describe_embedding(self) -> Space:
        low = (
            [0]* 9 +            # action_mask
            [-1] * 4 +          # real power
            [0] * 72 +         # my team types
            [0] * 72           # opponent team types
        )
        high = (
            [1] * 9 +           # action_mask
            [3] * 4 +           # real power
            [1] * 72 +         # my team types
            [1] * 72           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action, battle):
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        #print(f"🔢 DQN → action={action} | #moves={len(moves)} | #switches={len(switches)} | total={total_actions}")

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            print("❌ Action hors bornes, choix d'un move aléatoire")
            return self.choose_random_move(battle)
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action

Hélas je laisse tomber le dynamic output pour le moment
-
